In [3]:
import pandas as pd
import numpy as np
import time


In [4]:
store = pd.HDFStore('model_data.h5')
store.keys()

['/annual_household_relocation_rates',
 '/blocks',
 '/edges',
 '/edges_simple',
 '/households',
 '/jobs',
 '/nodes',
 '/nodes_simple',
 '/non_residential_units',
 '/nonres_rents',
 '/persons',
 '/places',
 '/region',
 '/residential_units',
 '/travel_data']

In [30]:
households = store['households']

households.head()

,serialno,persons,cars,income,race_of_head,age_of_head,workers,children,tenure,recent_mover,block_id
b'household_id',,,,,,,,,,,
1,2.012001e+12,1,1,110000,1,53,1,0,1,0,170310101001003
2,2.012001e+12,1,1,100000,1,49,1,0,1,0,170310101001000
3,2.009000e+12,6,3,130000,2,37,2,4,1,0,170310101001000
4,2.012000e+12,2,1,129130,1,62,2,0,1,0,170310101001003
5,2.013001e+12,2,1,105000,1,32,2,0,2,0,170310101001003


In [6]:
households.index.name = 'household_id'
households_reset = households.reset_index()
households_reset.head()

,household_id,serialno,persons,cars,income,race_of_head,age_of_head,workers,children,tenure,recent_mover,block_id
0,1,2.012001e+12,1,1,110000,1,53,1,0,1,0,170310101001003
1,2,2.012001e+12,1,1,100000,1,49,1,0,1,0,170310101001000
2,3,2.009000e+12,6,3,130000,2,37,2,4,1,0,170310101001000
3,4,2.012000e+12,2,1,129130,1,62,2,0,1,0,170310101001003
4,5,2.013001e+12,2,1,105000,1,32,2,0,2,0,170310101001003


In [35]:
persons = store["persons"]
persons.index.name = 'person_id'

persons_reset = persons.reset_index()


In [7]:
res_units = store["residential_units"]
res_units_reset = res_units.reset_index()
res_units_reset.head()

,b'du_id',block_id,year_built,building_type_id
0,1,170318042011000,1975,2
1,2,170318042011000,1955,1
2,3,170318042011000,1955,1
3,4,170318042011000,1955,1
4,5,170318042011000,1955,1


In [8]:
blocks = store["blocks"]
blocks_reset = blocks.reset_index()
blocks_reset.head()

,block_id,x,y,res_rents,res_values,square_meters_land,puma10_id,residential_unit_capacity,employment_capacity,rent_impute,value_impute,place_id,taz_id,zone_id
0,170310104003000,-87.655356,42.005923,779.0,280000.0,0,1703501,0,0,1,1,,1703144566507,-1
1,170310301014000,-87.651730,41.991317,693.0,191400.0,0,1703501,0,0,1,1,,1703144586490,-1
2,170310301021000,-87.651981,41.993331,746.0,245000.0,0,1703501,0,0,1,1,,1703144576492,-1
3,170310301032000,-87.652331,41.995379,820.0,130800.0,0,1703501,0,0,1,1,,1703144576494,-1
4,170310301041000,-87.652638,41.997481,1003.0,118500.0,0,1703501,0,0,1,1,,1703144576496,-1


#### Household Variables

In [130]:
def sort_incomes(df):
    HHINC_LESS20K = sum(df["income"] < 20000)
    HHINC_20TO35K = sum((df["income"] > 20000) & (df["income"] <= 35000))
    HHINC_35TO50K = sum((df["income"] > 35000) & (df["income"] <= 50000))
    HHINC_50TO75k = sum((df["income"] > 50000) & (df["income"] <= 75000))
    HHINC_75TO100K = sum((df["income"] > 75000) & (df["income"] <= 100000))
    HHINC_100TO150K = sum((df["income"] > 100000) & (df["income"] <= 150000))
    HHINC_OVER150K = sum(df["income"] >= 150000)
    return [HHINC_LESS20K, HHINC_20TO35K,HHINC_35TO50K,HHINC_50TO75k, HHINC_75TO100K,
           HHINC_100TO150K, HHINC_OVER150K]

In [15]:
def num_vehicles(df):
    NVEH_0 = sum(df["cars"] == 0)
    NVEH1 = sum(df["cars"] == 1)
    NVEH2 = sum(df["cars"] == 2)
    NVEH_3 = sum(df["cars"] >= 3)
    return [NVEH_0, NVEH1, NVEH2, NVEH_3]

In [128]:
def hh_size(df):
    HHSIZE_1 = sum(df["persons"] == 1)
    HHSIZE_2 = sum(df["persons"] == 2)
    HHSIZE_3 = sum(df["persons"] == 3)
    HHSIZE_4 = sum(df["persons"] == 4)
    HHSIZE_5 = sum(df["persons"] == 5)
    HHSIZE_6 = sum(df["persons"] == 6)
    HHSIZE_7 = sum(df["persons"] == 7)
    return [HHSIZE_1,HHSIZE_2,HHSIZE_3,HHSIZE_4,HHSIZE_5,HHSIZE_6,HHSIZE_7]

In [17]:
# def hh_size(df):
#     HUTYPE_SF = 
#     HUTYPE_2TO9UNITS = 
#     HUTYPE_10_UNITS = 
#     HUTYPE_OTHER = 

In [131]:
start = time.time()
df_lol = []
blk_id_list = []
for blk_id, blk_df in households_reset.head(100000).groupby("block_id"):
    blk_id_list.append(blk_id)
    row = []
#     display(blk_df) #create a list that is a row, populate that list and append it to a list set up before the 4 loop
    row.extend(sort_incomes(blk_df)) # returns list with counts of households in each income bin
    row.extend(hh_size(blk_df))
    row.extend(num_vehicles(blk_df))
    df_lol.append(row)

households_df = pd.DataFrame(df_lol, columns=["HHINC_LESS20K", "HHINC_20TO35K", "HHINC_35TO50K", 
        "HHINC_50TO75k", "HHINC_75TO100K", "HHINC_100TO150K", "HHINC_OVER150K", "HHSIZE_1", "HHSIZE_2",
        "HHSIZE_3", "HHSIZE_4", "HHSIZE_5", "HHSIZE_6", "HHSIZE_7",
        "NVEH_0", "NVEH1", "NVEH2", "NVEH_3"], index=blk_id_list)

end = time.time()



In [135]:
elapsed = end- start
elapsed
households_df.head()


,HHINC_LESS20K,HHINC_20TO35K,HHINC_35TO50K,HHINC_50TO75k,HHINC_75TO100K,HHINC_100TO150K,HHINC_OVER150K,HHSIZE_1,HHSIZE_2,HHSIZE_3,HHSIZE_4,HHSIZE_5,HHSIZE_6,HHSIZE_7,NVEH_0,NVEH1,NVEH2,NVEH_3
170310101001000,36,10,8,4,6,3,0,52,13,0,3,0,1,0,25,31,11,2
170310101001001,7,3,6,0,2,4,0,13,5,0,3,2,0,0,9,8,6,0
170310101001002,8,3,3,2,0,4,0,15,5,0,1,1,0,0,6,12,4,0
170310101001003,59,39,20,18,5,11,0,110,26,0,12,3,1,1,68,73,11,1
170310101002000,15,12,3,8,2,12,0,11,10,15,9,5,1,0,19,23,8,2


#### Person-level variables

In [108]:
def gender(df):
    SEX_MALE = len(df[df["sex"] == 1])
    SEX_FEMALE = len(df[df["sex"] == 2])
    return [SEX_MALE, SEX_FEMALE]

In [146]:
def age(df):
    AGE_under15 = len(df["age"] < 15)
    AGE_over_40 = len(df["age"] > 40)
    return [AGE_under15, AGE_over_40]
#     AGE_15to24 = len((df["age"] <= 24) & (df["age"] > 15))
#     print("15 to 24:", AGE_15to24)
#     AGE_25to34 = len((df["age"] <= 34) & (df["age"] > 25))
#     print("25 to 34:", AGE_25to34)
#     AGE_35to44 = len(df["age"] <= 44 & (df["age"] > 35))
#     print("35 to 44:", AGE_35to44)
#     AGE_45to54 = len(df["age"] <= 54 & (df["age"] > 45))
#     AGE_55to64 = len(df["age"] <= 64 & (df["age"] > 55))
#     AGE_65plus = len(df["age"] >=65)
# #     print("age", [AGE_under15, AGE_15to24, AGE_25to34, AGE_35to44, AGE_45to54, AGE_55to64, AGE_65plus])
#     return [AGE_under15, AGE_15to24, AGE_25to34, AGE_35to44, AGE_45to54, AGE_55to64, AGE_65plus]

In [120]:
persons_and_households = pd.merge(persons_reset, households_reset, how="inner", left_on="household_id", right_on="household_id")
persons_and_households.head()

,person_id,age,earning,edu,hours,household_id,member_id,race_id,relate,sex,...,persons,cars,income,race_of_head,age_of_head,workers,children,tenure,recent_mover,block_id
0,1,61,28000,21,40,169,1,1,0,2,...,1,1,28000,1,61,1,0,1,0,170310101001000
1,2,61,0,13,0,101,1,1,0,2,...,1,0,8270,1,61,0,0,2,0,170310101001003
2,3,63,0,22,0,125,1,1,0,2,...,1,1,7800,1,63,0,0,2,0,170310101001000
3,4,63,0,22,0,126,1,1,0,2,...,1,1,7800,1,63,0,0,2,0,170310101001003
4,5,63,0,22,0,127,1,1,0,2,...,1,1,7800,1,63,0,0,2,0,170310101001000


In [136]:
# persons_reset.head()

In [117]:
def race(df):
    RACE_WHITE = len(df[df["race_id"] == 1])
    RACE_BLACK = len(df[df["race_id"] == 2])
    RACE_INDIAN = len(df[df["race_id"] == 3])
    RACE_ASIAN = len(df[df["race_id"] == 6])
    RACE_OTHER = len(df[df["race_id"] == 8])
#     RACE_HISPANIC = len(df["race_id"] == ) I dont think there is information in the UrbanSim data about hispanic origin
    return [RACE_WHITE, RACE_BLACK, RACE_INDIAN, RACE_ASIAN, RACE_OTHER]

In [153]:
start = time.time()
df_lol_persons = []
blk_id_list_persons = []
for blk_id_p, blk_df_p in persons_and_households.head(500).groupby("block_id"):
#     display(blk_df_p)
    print(blk_df_p.dtypes)
    blk_id_list_persons.append(blk_id_p)
    row = []
    row.extend(gender(blk_df_p))
    row.extend(race(blk_df_p))
    row.extend(age(blk_df_p))    
    df_lol_persons.append(row)
    
    
persons_df = pd.DataFrame(df_lol_persons, columns=["SEX_MALE", "SEX_FEMALE", "RACE_WHITE",
              "RACE_BLACK", "RACE_INDIAN", "RACE_ASIAN", "RACE_OTHER","AGE_under15", "AGE_over40"], index=blk_id_list_persons)
# persons_df = pd.DataFrame(df_lol_persons, columns=["AGE_under15", "AGE_15to24", "AGE_25to34", "AGE_35to44",
#     "AGE_45to54","AGE_55to64","AGE_65plus", "SEX_MALE", "SEX_FEMALE", "RACE_WHITE",
#              "RACE_BLACK", "RACE_INDIAN", "RACE_ASIAN", "RACE_OTHER"], index=blk_id_list_persons)
print("here")
end = time.time()



person_id         int64
age               int64
earning           int64
edu               int64
hours             int64
household_id      int64
member_id         int64
race_id           int64
relate            int64
sex               int64
student           int64
work_at_home      int64
worker            int64
serialno        float64
persons           int64
cars              int64
income            int64
race_of_head      int64
age_of_head       int64
workers           int64
children          int64
tenure            int64
recent_mover      int64
block_id         object
dtype: object
person_id         int64
age               int64
earning           int64
edu               int64
hours             int64
household_id      int64
member_id         int64
race_id           int64
relate            int64
sex               int64
student           int64
work_at_home      int64
worker            int64
serialno        float64
persons           int64
cars              int64
income            int64
ra

In [151]:
persons_df.head(50)


,SEX_MALE,SEX_FEMALE,RACE_WHITE,RACE_BLACK,RACE_INDIAN,RACE_ASIAN,RACE_OTHER,AGE_under15,AGE_over40
170310101001000,46,50,54,32,0,5,0,96,96
170310101001001,18,27,19,18,0,2,0,45,45
170310101001002,16,18,18,15,0,0,0,34,34
170310101001003,116,122,94,131,1,4,1,238,238
170310101002000,1,1,2,0,0,0,0,2,2
170310101002001,0,1,1,0,0,0,0,1,1
170310101002002,0,2,2,0,0,0,0,2,2
170310101002005,1,2,3,0,0,0,0,3,3
170310101002006,0,1,1,0,0,0,0,1,1
170310101002008,1,1,2,0,0,0,0,2,2


#### Land Use

In [ ]:
LUcodes = pd.read_csv('land_use_codes_csv.csv')
